# Úkol č. 1 - část web scraping


## Výběr zdroje dat

Vybrala jsem si první katalog:

  * [Vysoká škola finanční a správní, a.s.](https://is.vsfs.cz/thesis/)
  
## Postup stahování a ukládání
V proměnné data jsou uloženy paramety vyhledávání.
Stahuji každý typ práce zvlášť vždy v cyklu podle počtu stránek jednotlivých prací.
Většinu údajů čtu rovnou ze stránky vyhledávání, pouze jména oponenta a vedoucího hledám až na samotné stránce každé práce.
Údaje ukládám vždy po 50 do jednotlivých csv souborů (složka CSV a dále složky BC, ING a MGR). Captchu jsem zadávala ručně
když bylo potřeba, cyklus requestu jsem na tyto chvíle zastavila input() polem.
Po načtení dostatečného množství dat, jsem z jednotlivých csv souborů vytvořila jeden, který obsahuje všechna data, se kterými dále pracuji v druhé a třet části úkolu.

CSV soubor je zde: ./CSV/data.csv


In [ ]:
import numpy as np
import pandas as pd

import requests
from bs4 import BeautifulSoup

In [ ]:
def innerHTML(element):
    """Returns the inner HTML of an element as a UTF-8 encoded bytestring"""
    return element.encode_contents().decode("utf-8")

def getLink(object):
    try:
        link = object.findAll('a')[0].get('href')
        return link
    except:
        return -1
        

url = 'https://is.vsfs.cz/thesis/?'
url_page = 'https://is.vsfs.cz'

data = {
    'FAK':'6410',
    'PRI' : '-',
    'ROK' : '-',
    'TIT' : '-', 
    'PRA' : '-',
    'vypsat':'1',
    'exppar' : '1',
    'por' : '0',
}
students = []

In [ ]:
def is_number(s):
    try:
        int(s)
        return True
    except ValueError:
        return False
    
def returnWork(student = object, page  = object, prace = object):
    work = {
        "schoolName": np.nan,
        "studentName": np.nan,
        "degree": np.nan,
        "year" : np.nan,
        "defended": np.nan,
        "type":np.nan,
        "program": np.nan,
        "major": np.nan,
        "title": np.nan,
        "supervisor": np.nan,
        "oponent": np.nan
    }

    divisions = page.findAll('div', { "class" : "oddil"})
    
    work['studentName'] = innerHTML(student.findAll('b')[0])
    if(innerHTML(student.findAll('i')[0]).find('roz.') != -1):
        index_i = 1
    else:
        index_i = 0
        
    work['schoolName'] = innerHTML(student.findAll('i')[index_i])

    icka = student.findAll('i')
    
    if(is_number(innerHTML(icka[index_i+1]))):
        work['year'] = int(innerHTML(icka[index_i+1]))
    if(innerHTML(icka[index_i+2]) == "úspěšně absolvováno"):
        work['defended'] = 1
        work['degree'] = innerHTML(icka[index_i+3])
    else:
        work['defended'] = 0
    
    work['program'] = innerHTML(icka[index_i+4])
    work['major'] = innerHTML(icka[index_i+5])
    work['type'] = prace
    if(len(icka) > 7+index_i):
        work['title'] = innerHTML(icka[-1])
    else:
        work['title'] = innerHTML(icka[-1])
    
    if(len(icka) < 7):
        work['title'] = np.nan
        
    
    
    #display(divisions)
    if(len(divisions) > 2):
        lines = divisions[2].find_all('li')
        headers = divisions[2].find_all('h4')
        #display(lines)
        if(len(headers) > 1 and innerHTML(headers[-2]) == "Vedoucí:"):
            string = innerHTML(lines[-2]).split(',')
            work['supervisor'] = ",".join(string[:-1])
        if(len(headers) > 1 and innerHTML(headers[-1]) == "Oponent:"):
            string = innerHTML(lines[-1]).split(',')
            work['oponent'] = ",".join(string[:-1])

    return work

theses = []


In [ ]:
#get data BC
prace = 'Bc.'

for pages in range(1351, 2402, 50):
    data['TIT'] = prace
    data['ROK'] ='-'
    data['por'] = pages
    display(data)
    r = requests.post(url, data)
    r.encoding='utf-8'
    soup = BeautifulSoup(r.text, "html.parser")
    divs = soup.findAll('div', style="margin-left:50px")
    index = 0
    deg = soup.new_tag("i")

    for div in divs:
        if(getLink(div) == -1):
            continue
        l = requests.post(url_page+getLink(div))
        l.encoding='utf-8'
        soup_page = BeautifulSoup(l.text, "html.parser")
        while(l.status_code != 200):
            print('max limit!')
            print(url_page+getLink(div))
            input()
            l = requests.post(url_page+getLink(div))
            l.encoding='utf-8'
            soup_page = BeautifulSoup(l.text, "html.parser")
        theses.append(returnWork(div, soup_page, prace))
    data_50 = pd.DataFrame(theses)
    data_50.to_csv("./CSV/BC/data_{}.csv".format(pages),sep=';')
    theses.clear()

print("done Bc")
display(len(theses))

In [ ]:
#get data MGR
prace = 'Mgr.'

for pages in range(1, 51, 50):
    data['TIT'] = prace
    data['ROK'] ='-'
    data['por'] = pages
    display(data)
    r = requests.post(url, data)
    r.encoding='utf-8'
    soup = BeautifulSoup(r.text, "html.parser")
    divs = soup.findAll('div', style="margin-left:50px")
    index = 0
    deg = soup.new_tag("i")

    for div in divs:
        if(getLink(div) == -1):
            continue
        l = requests.post(url_page+getLink(div))
        l.encoding='utf-8'
        soup_page = BeautifulSoup(l.text, "html.parser")
        while(l.status_code != 200):
            print('max limit!')
            print(url_page+getLink(div))
            input()
            l = requests.post(url_page+getLink(div))
            l.encoding='utf-8'
            soup_page = BeautifulSoup(l.text, "html.parser")
        theses.append(returnWork(div, soup_page, prace))
    data_50 = pd.DataFrame(theses)
    data_50.to_csv("./CSV/MGR/data_{}.csv".format(pages),sep=';')
    theses.clear()

print("done Mgr")
display(len(theses))

In [ ]:
#get data ING
prace = 'Ing.'

for pages in range(1101, 2002, 50):
    data['TIT'] = prace
    data['ROK'] ='-'
    data['por'] = pages
    display(data)
    r = requests.post(url, data)
    r.encoding='utf-8'
    soup = BeautifulSoup(r.text, "html.parser")
    divs = soup.findAll('div', style="margin-left:50px")
    index = 0
    deg = soup.new_tag("i")

    for div in divs:
        if(getLink(div) == -1):
            continue
        l = requests.post(url_page+getLink(div))
        l.encoding='utf-8'
        soup_page = BeautifulSoup(l.text, "html.parser")
        while(l.status_code != 200):
            print('max limit!')
            print(url_page+getLink(div))
            input()
            l = requests.post(url_page+getLink(div))
            l.encoding='utf-8'
            soup_page = BeautifulSoup(l.text, "html.parser")
        theses.append(returnWork(div, soup_page, prace))
    data_50 = pd.DataFrame(theses)
    data_50.to_csv("./CSV/ING/data_{}.csv".format(pages),sep=';')
    theses.clear()

print("done Ing")
display(len(theses))

In [ ]:
#concat data into one CSV file
import glob

filesBC = glob.glob('CSV/BC/*.csv')
filesING = glob.glob('./CSV/ING/*.csv')
filesMGR = glob.glob('./CSV/MGR/*.csv')
filesBC
dfBC =pd.concat([pd.read_csv(f, sep=';') for f in filesBC], ignore_index=True)

dfING =pd.concat([pd.read_csv(f, sep=';') for f in filesING], ignore_index=True)
dfMGR =pd.concat([pd.read_csv(f, sep=';') for f in filesMGR], ignore_index=True)
df = pd.concat([dfBC,dfMGR,dfING], ignore_index=True)
df = df.drop("Unnamed: 0", axis=1)
#save file
df.to_csv("./CSV/data.csv",sep=';')